In [2]:
import simpy

SimPy is a discrete-event simulation library. The behavior of active components (like vehicles, customers or messages) is modeled with processes. All processes live in an environment. They interact with the environment and with each other via events.

Processes are described by simple Python [generators](https://docs.python.org/3/reference/expressions.html#yieldexpr). You can call them process function or process method, depending on whether it is a normal function or method of a class. During their lifetime, they create events and yield them in order to wait for them to be triggered.

When a process yields an event, the process gets suspended. SimPy resumes the process, when the event occurs (we say that the event is triggered). Multiple processes can wait for the same event. SimPy resumes them in the same order in which they yielded that event.

An important event type is the `Timeout`. Events of this type are triggered after a certain amount of (simulated) time has passed. They allow a process to sleep (or hold its state) for the given time. A `Timeout` and all other events can be created by calling the appropriate method of the `Environment` that the process lives in (`Environment.timeout()` for example).

## First Process

Our first example will be a car process. The car will alternately drive and park for a while. When it starts driving (or parking), it will print the current simulation time.

In [1]:
def car(env):
     while True:
            print('Start parking at %d' % env.now)
            parking_duration = 5
            yield env.timeout(parking_duration)
            
            print('Start driving at %d' % env.now)
            trip_duration = 2
            yield env.timeout(trip_duration)

Our car process requires a reference to an `Environment` (env) in order to create new events. The car’s behavior is described in an infinite loop. Remember, this function is a generator. Though it will never terminate, it will pass the control flow back to the simulation once a yield statement is reached. Once the yielded event is triggered (“it occurs”), the simulation will resume the function at this statement.

As I said before, our car switches between the states parking and driving. It announces its new state by printing a message and the current simulation time (as returned by the `Environment.now` property). It then calls the `Environment.timeout()` factory function to create a `Timeout` event. This event describes the point in time the car is done parking (or driving, respectively). By yielding the event, it signals the simulation that it wants to wait for the event to occur.

Now that the behavior of our car has been modeled, lets create an instance of it and see how it behaves:

In [3]:
env = simpy.Environment()

In [4]:
env.process(car(env))

<Process(car) object at 0x7fb1c0409790>

In [5]:
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


## Process Interaction

The `Process` instance that is returned by `Environment.process()` can be utilized for process interactions. The two most common examples for this are to wait for another process to finish and to interrupt another process while it is waiting for an event.

## Waiting for a Process

As it happens, a SimPy `Process` can be used like an event (technically, a process actually is an event). If you yield it, you are resumed once the process has finished. Imagine a car-wash simulation where cars enter the car-wash and wait for the washing process to finish. Or an airport simulation where passengers have to wait until a security check finishes.

Lets assume that the car from our last example magically became an electric vehicle. Electric vehicles usually take a lot of time charging their batteries after a trip. They have to wait until their battery is charged before they can start driving again.

We can model this with an additional `charge()` process for our car. Therefore, we refactor our car to be a class with two process methods: `run()` (which is the original `car()` process function) and `charge()`.

The `run` process is automatically started when `Car` is instantiated. A new `charge` process is started every time the vehicle starts parking. By yielding the `Process` instance that `Environment.process()` returns, the `run` process starts waiting for it to finish:

In [22]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            # parking
            print('start parking \t @ %d' % self.env.now)
            parking_duration = 3
            yield self.env.process(self.park(parking_duration))
            
            # charging
            print('start charging \t @ %d' % self.env.now)
            charging_duration = 19
            yield self.env.process(self.charge(charging_duration))
            
            # driving
            print('start driving \t @ %d' % self.env.now)
            trip_duration = 41
            yield self.env.process(self.drive(trip_duration))
            
            
    def park(self, duration):
        yield self.env.timeout(duration)
        
    def charge(self, duration):
        yield self.env.timeout(duration)
        
    def drive(self, duration):
        yield self.env.timeout(duration)

In [23]:
env = simpy.Environment()

In [24]:
car = Car(env)

In [25]:
env.run(until = 210)

start parking 	 @ 0
start charging 	 @ 3
start driving 	 @ 22
start parking 	 @ 63
start charging 	 @ 66
start driving 	 @ 85
start parking 	 @ 126
start charging 	 @ 129
start driving 	 @ 148
start parking 	 @ 189
start charging 	 @ 192


## Interrupting Another Process

SimPy allows you to interrupt a running process by calling its interrupt() method:

In [32]:
def empty_charge(env, car):
    yield env.timeout(10)
    car.action.interrupt()

In [33]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.charge_level = 100
        
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            # parking
            print('start parking \t @ %d' % self.env.now)
            parking_duration = 3
            yield self.env.process(self.park(parking_duration))
            
            # charging
            try:
                print('start charging \t @ %d' % self.env.now)
                charging_duration = 19
                yield self.env.process(self.charge(charging_duration))
            except simpy.Interrupt:
                print('Was interrupted. Hope, the battery is full enough ...')
            
            # driving
            print('start driving \t @ %d' % self.env.now)
            trip_duration = 41
            yield self.env.process(self.drive(trip_duration))
            
            
    def park(self, duration):
        yield self.env.timeout(duration)
        
    def charge(self, duration):
        yield self.env.timeout(duration)
        
    def drive(self, duration):
        yield self.env.timeout(duration)
        
        
    # method to record charge level of car
    def charge_level(self)

In [34]:
env = simpy.Environment()

In [35]:
car = Car(env)

In [36]:
env.process(empty_charge(env, car))

<Process(empty_charge) object at 0x7fb1c0a63cd0>

In [37]:
env.run(until=210)

start parking 	 @ 0
start charging 	 @ 3
Was interrupted. Hope, the battery is full enough ...
start driving 	 @ 10
start parking 	 @ 51
start charging 	 @ 54
start driving 	 @ 73
start parking 	 @ 114
start charging 	 @ 117
start driving 	 @ 136
start parking 	 @ 177
start charging 	 @ 180
start driving 	 @ 199
